In [2]:
import pandas as pd
from tqdm.notebook import tqdm
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import sklearn
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler,StandardScaler

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
data = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/Hakaton/statistics-07-20.csv', sep = ';')

# Анализ данных

In [6]:
data.head()

,index_train,length,car_number,destination_esr,adm,danger,gruz,loaded,operation_car,operation_date,...,operation_st_id,operation_train,receiver,rodvag,rod_train,sender,ssp_station_esr,ssp_station_id,tare_weight,weight_brutto
0,NaN,NaN,62827035,984700.0,NaN,NaN,NaN,2.0,81.0,2020-07-16 03:40:00,...,2.000039e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,1.0,62827035,862201.0,20.0,NaN,421034.0,NaN,18.0,2020-07-16 14:10:00,...,2.002026e+09,NaN,93149858.0,60.0,NaN,68398528.0,NaN,NaN,249.0,NaN
2,NaN,NaN,62827316,863007.0,NaN,NaN,NaN,2.0,80.0,2020-07-16 14:50:00,...,2.001933e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,62827381,862108.0,NaN,NaN,NaN,2.0,80.0,2020-07-16 14:16:00,...,2.001931e+09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,1.0,62845730,887904.0,20.0,NaN,421034.0,NaN,19.0,2020-07-16 15:15:00,...,2.000039e+09,NaN,81195103.0,60.0,NaN,52682351.0,NaN,NaN,248.0,NaN


In [7]:
# # функция для поиска пустых данных
def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])

In [8]:
# # функция для поиска дубликатов по столбцам
def find_duplicates_by_columns(df):  
    duplicates = []
    column = []
    for i in df.columns:
        duplicates.append(df[i].duplicated().sum())
        column.append(i)
    duplicates = pd.Series(duplicates) 
    column = pd.Series(column) 
    return pd.concat([column, duplicates], axis = 1,
                     keys = ['column', 'duplicates']).sort_values(by = 'duplicates',ascending = False).head(40)

In [9]:
#Поиск пустых данных
missing_data(data)

,Total,Percent
danger,4077929,97.327249
tare_weight,3085197,73.633880
adm,3085197,73.633880
rod_train,2623663,62.618526
weight_brutto,2623318,62.610292
ssp_station_id,2255185,53.824123
operation_train,2222194,53.036732
ssp_station_esr,2222193,53.036708
index_train,2222193,53.036708
gruz,1779461,42.470098


Error: Runtime no longer has a reference to this dataframe, please re-run this cell and try again.


In [10]:
#Поиск дубликатов
find_duplicates_by_columns(data)

,column,duplicates
5,danger,4189913
7,loaded,4189911
14,rodvag,4189903
4,adm,4189901
12,operation_train,4189901
8,operation_car,4189894
15,rod_train,4189887
1,length,4189838
19,tare_weight,4189580
11,operation_st_id,4189170


# Предобработка

In [11]:
data.operation_date = pd.to_datetime(data.operation_date, format='%Y-%m-%d %H:%M')
data.operation_date

0         2020-07-16 03:40:00
1         2020-07-16 14:10:00
2         2020-07-16 14:50:00
3         2020-07-16 14:16:00
4         2020-07-16 15:15:00
                  ...        
4189910   2020-07-16 03:29:00
4189911   2020-07-16 03:30:00
4189912   2020-07-15 20:00:00
4189913   2020-07-16 02:37:00
4189914   2020-07-16 04:28:00
Name: operation_date, Length: 4189915, dtype: datetime64[ns]

In [12]:
#заполнениe нулевых значений в наборе данных
#Признак опасного груза в вагоне

data.danger = data.danger.fillna(0)

In [13]:
#узнаем размерность
data.shape

(4189915, 21)

In [14]:
data.danger = data.danger.fillna(0)
data = data.sort_values(by = ['operation_date'], ascending = True)
#пересортируем строки (drop=True - для удаления старых индексов)
data = data.reset_index(drop = True)

# Получение всех подходящих условию операций включения вагонов в поезд для всех вагонов на всех станциях

In [15]:
# возвращает список датафреймов со всеми операцифми по каждому уникальному вагону 
def get_data_vags(data):
    
    data_vags_stack = [] 
    vags_numbers = data['car_number'].unique()
    print('Собираются датафреймы для каждого из {} уникальных вагонов :'.format(len(vags_numbers)))
    for vags_number in tqdm(vags_numbers):
        vag_df = pd.DataFrame(data[data.car_number == vags_number].sort_values(by = ['operation_date'], ascending = True))
        vag_df = vag_df.reset_index(drop=True)
        data_vags_stack.append(vag_df)
        
    return data_vags_stack #список датафрейфмов

In [16]:
# принимает датафрейм со всеми операцифми по каждому уникальному вагону
# возвращат список датафреймов, где каждый элемент - датафрейм одного уникального вагона на одной уникальной станции
def split_data_vag_into_stations(data_vag):  
    
    data_vags_st_stack = []
    st_esrS = data_vag.operation_st_esr.unique()
    for st_esr in st_esrS:
        vag_st_df = pd.DataFrame(data_vag[data_vag.operation_st_esr == st_esr].sort_values(by = ['operation_date'], ascending = True))
        vag_st_df = vag_st_df.reset_index(drop = True)
        data_vags_st_stack.append(vag_st_df)
    
    return data_vags_st_stack # стек датафреймов одного вагона на разных станциях

Для каждого датафрейма с данными об уникальном вагоне на уникальной станции за все время, ищутся operation_train in ([ 2., 62., 22., 42.] ), идем по наблюдениям назад по времени и ищем первую встречную operation_car == 4, идем по наблюдениям назад по времени до предыдущего отправления в operation_train и если в интервале есть операции погрузки/выгрузки до текущей операции включения, значит данную операцию включения в поезд берем, если же в интервале нет операций погрузки выгрузки, то переходим к предыдущей операции включения и проверяем её интервал до отправления на наличие операций погрузки/выгрузки. Из нескольких подходящих вышеуказанному условию наблюдений внутри каждого интервала (между отправлениями поезда), в которых операция с вагоном == операция включения в поезд, берем только одну последнюю операцию включения в поезд.


In [17]:
# принимает датафрейм с данными об уникальном вагоне на уникальной станции за все время
# возвращает датафрейм с операциями включения, подходящими условию, для одного уникального вагона на одной уникальной станции
# operation_car Код операции ВМ  АСОУП
# operation_train Код операции ПМ  АСОУП
def get_includes(data_vag_st):
    
    buffer = []
    for i in reversed(range(len(data_vag_st))): # идем по последнним наблюдениям к боллее старым для каждого датафрейма с данными об уникальном вагоне на уникальной станции за все время
        if (data_vag_st.loc[i, 'operation_train'] in (2, 62, 22, 42)) & (i>0): # находим операцию отправления поезда
        # условие operation_car пусто, operation_train не пусто
        #https://rts-nn.ru/info/
        ##https://cargolk.rzd.ru/catalogs/cargo_operations
        #https://online.freicon.ru/info/wagon-operations?page=3&perPage=25
            j = i-1
            valid_inclusion_operations_indexes = [] #подходящие вагоны с операцией включения
            while (data_vag_st.loc[j, 'operation_train'] not in (2, 62, 22, 42)) & (j>0):# ограничиваем поиск операции включения предыдущей операцией отправления поезда
                if data_vag_st.loc[j, 'operation_car'] == 4:  #находим операции включения в поезд перед последней операцией отправления
                    flag = False
                    k = j-1
                    while (data_vag_st.loc[k, 'operation_train'] not in (2, 62, 22, 42)) & (k>=0): #ограничиваем интервал до предыдущей операции отправления поезда
                        if data_vag_st.loc[k, 'operation_car'] in (10,11,13,14,15,17,18,19,20,21,23,24,25,28,29):#проверяем наличие хоть одной операции разгрузки/погрузки
                        # цифры взяты из предложенного селекта ((kop_vmd >= 10 and kop_vmd <= 19) or (kop_vmd >= 20 and kop_vmd <= 29)) and date_op<=DATE('2021-08-31') and date_op>=DATE('2021-08-01')
                            flag = True 
                        if k > 0:
                            k -= 1                # смотрим предыдущие записиси в интервале
                        else:
                            break  
                    if flag == True:
                        valid_inclusion_operations_indexes.append(j)
                j-=1
                
            if valid_inclusion_operations_indexes : buffer.append(data_vag_st.iloc[valid_inclusion_operations_indexes[0]]) # список может быть пустым, поэтому есть данное условие
            
    return pd.DataFrame(data = buffer).reset_index(drop = True)

In [18]:
# возвращает стек непустых датафреймов с операциями включения,соответствующих условию, для каждого вагона на всех станциях
def get_all_includes_by_vags(data):
    data_vags_stack = get_data_vags(data)
    includes_by_vags = []
    print('Идет поиск необходимых операций включений для каждого уникального вагона из {} :'.format(len(data_vags_stack)))
    for data_vag in tqdm(data_vags_stack):
        data_vag_stations_stack = split_data_vag_into_stations(data_vag)
        includes_by_vag_st = [] # те, что получились здесь с разных станций сложим в один датафрейм ( для каждого вагона)
        for data_vag_station in data_vag_stations_stack:
            includes_by_vag_st.append(get_includes(data_vag_station))
            
        includes_by_vag_df = pd.concat([x for x in includes_by_vag_st]) # соединим датафреймы с каждой отдельной станцей в один для каждого вагона
        includes_by_vags.append(includes_by_vag_df)
    return [df for df in includes_by_vags if not df.empty]

In [19]:
# соединяет датафреймы друг с другом
def get_includes_df(all_includes_by_vags):
    df_includes = pd.concat([x for x in all_includes_by_vags])
    print('Общее по всем вагонам количество подходящих условию операций включения составило {}'.format(df_includes.shape[0]))
    return df_includes

In [21]:
%%time
all_includes_by_vags = get_all_includes_by_vags(data)

Собираются датафреймы для каждого из 441248 уникальных вагонов :


  0%|          | 0/441248 [00:00<?, ?it/s]

Идет поиск необходимых операций включений для каждого уникального вагона из 441248 :


  0%|          | 0/441248 [00:00<?, ?it/s]

CPU times: user 2h 19min 3s, sys: 3min 15s, total: 2h 22min 18s
Wall time: 1h 55min 52s


In [22]:
df_includes = get_includes_df(all_includes_by_vags)

Общее по всем вагонам количество подходящих условию операций включения составило 501850


In [23]:
df_includes.to_csv('/content/drive/MyDrive/ColabNotebooks/Hakaton/clear_data.csv')